In [1]:
import pandas as pd
import numpy as np

In [2]:
bus_stops = pd.read_csv("datasets/1.-Transportation/bus-stops.csv")
metro_stops = pd.read_csv("datasets/1.-Transportation/public-transport.csv")

In [3]:
# Sacamos tabla de bus
bus_Z = bus_stops.groupby("District.Name").count()
bus_W = bus_Z[["Transport","Code"]]
bus_W.reset_index(drop=True, inplace=True)

distritos = ['Ciutat Vella','Eixample','Gràcia',
             'Horta-Guinardó','Les Corts','Nou Barris',
             'Sant Andreu','Sant Martí','Sants-Montjuic','Sarrià-Sant Gervasi']
bus_W["District.Name"] = distritos

<ipython-input-3-9994e0433218>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus_W["District.Name"] = distritos


In [4]:
# Tabla de metro
metro_Z = pd.DataFrame({"District.Name" : ['Ciutat Vella','Eixample','Gràcia','Horta-Guinardó',
                                           'Les Corts','Nou Barris', 'Sant Andreu','Sant Martí',
                                           'Sants-Montjuic','Sarrià-Sant Gervasi'],
                          "Stops" : [22,103,15,31,15,38,36,48,39,25]})

In [5]:
# Merge de bus y metro en misma tabla
transport_stops = bus_W.merge(right=metro_Z, how = "left")
result_transport = transport_stops[["District.Name", "Transport","Stops"]].copy()
result_transport.columns = ["Nom_districte", "Bus", "Metro"]

In [6]:
# Merge de areas a tabla de bus y metro
areas = pd.DataFrame({"Nom_districte" : ['Ciutat Vella','Eixample','Gràcia',
                                         'Horta-Guinardó','Les Corts','Nou Barris',
                                         'Sant Andreu','Sant Martí','Sants-Montjuic',
                                         'Sarrià-Sant Gervasi'],
                          "Superficie" : [4.37,7.46,4.19,11.9,6.08,8,6.56,10.8,21.35,20.09]})

stops_areas = result_transport.merge(right=areas, how = "left")
stops_areas["Nº parades bus/km2"] = stops_areas["Bus"] / stops_areas["Superficie"]
stops_areas["Nº parades metro/km2"] = stops_areas["Metro"] / stops_areas["Superficie"]
stops_areas

,Nom_districte,Bus,Metro,Superficie,Nº parades bus/km2,Nº parades metro/km2
0,Ciutat Vella,167,22,4.37,38.215103,5.034325
1,Eixample,405,103,7.46,54.289544,13.806971
2,Gràcia,210,15,4.19,50.119332,3.579952
3,Horta-Guinardó,389,31,11.90,32.689076,2.605042
4,Les Corts,207,15,6.08,34.046053,2.467105
5,Nou Barris,330,38,8.00,41.250000,4.750000
6,Sant Andreu,228,36,6.56,34.756098,5.487805
7,Sant Martí,356,48,10.80,32.962963,4.444444
8,Sants-Montjuic,415,39,21.35,19.437939,1.826698
9,Sarrià-Sant Gervasi,439,25,20.09,21.851667,1.244400


In [7]:
# Tabla de transporte público / area absoluta:
stops_abs = stops_areas[["Nom_districte", "Bus", "Metro", "Superficie"]]
stops_abs

,Nom_districte,Bus,Metro,Superficie
0,Ciutat Vella,167,22,4.37
1,Eixample,405,103,7.46
2,Gràcia,210,15,4.19
3,Horta-Guinardó,389,31,11.90
4,Les Corts,207,15,6.08
5,Nou Barris,330,38,8.00
6,Sant Andreu,228,36,6.56
7,Sant Martí,356,48,10.80
8,Sants-Montjuic,415,39,21.35
9,Sarrià-Sant Gervasi,439,25,20.09


In [8]:
def color_abs(val):
    if val > 410:
        color = "green"
    elif val < 208:
        color = "red"
    else:
        color = "grey"
    return 'color: %s' % color

def color_abs_metro(val):
    
    if val > 100:
        color = "green"
    elif val < 20:
        color = "red"
    else:
        color = "grey"
    return 'color: %s' % color

In [14]:
abs_colors = stops_abs.style.applymap(color_abs, subset=pd.IndexSlice[:, ["Bus"]]).applymap(color_abs_metro, subset=pd.IndexSlice[:, ["Metro"]])
abs_colors.format({"Superficie": lambda x:round(x, 2)})

,Nom_districte,Bus,Metro,Superficie
0,Ciutat Vella,167,22,4.37
1,Eixample,405,103,7.46
2,Gràcia,210,15,4.19
3,Horta-Guinardó,389,31,11.9
4,Les Corts,207,15,6.08
5,Nou Barris,330,38,8.0
6,Sant Andreu,228,36,6.56
7,Sant Martí,356,48,10.8
8,Sants-Montjuic,415,39,21.35
9,Sarrià-Sant Gervasi,439,25,20.09


In [10]:
# Tabla de transporte público / area relativa:
stops_rel = stops_areas[["Nom_districte", "Nº parades bus/km2", "Nº parades metro/km2"]].copy()
stops_rel["Nº parades metro/km2"] = stops_rel["Nº parades metro/km2"].apply(lambda x: round(x, 2))
stops_rel["Nº parades bus/km2"] = stops_rel["Nº parades bus/km2"].apply(lambda x: round(x, 2))
stops_rel

,Nom_districte,Nº parades bus/km2,Nº parades metro/km2
0,Ciutat Vella,38.22,5.03
1,Eixample,54.29,13.81
2,Gràcia,50.12,3.58
3,Horta-Guinardó,32.69,2.61
4,Les Corts,34.05,2.47
5,Nou Barris,41.25,4.75
6,Sant Andreu,34.76,5.49
7,Sant Martí,32.96,4.44
8,Sants-Montjuic,19.44,1.83
9,Sarrià-Sant Gervasi,21.85,1.24


In [11]:
def color_rel(val):
    
    if val > 50:
        color = "green"
    elif val < 22:
        color = "red"
    else:
        color = "grey"
    return 'color: %s' % color

def color_rel_metro(val):
    
    if val > 10:
        color = "green"
    elif val < 2:
        color = "red"
    else:
        color = "grey"
    return 'color: %s' % color

In [12]:
rel_colors = stops_rel.style.applymap(color_rel, subset=pd.IndexSlice[:, ["Nº parades bus/km2"]]).applymap(color_rel_metro, subset=pd.IndexSlice[:, ["Nº parades metro/km2"]])

rel_colors.format({"Nº parades bus/km2": lambda y:round(y, 2),
                   "Nº parades metro/km2": lambda z:round(z, 2)
                  })

,Nom_districte,Nº parades bus/km2,Nº parades metro/km2
0,Ciutat Vella,38.22,5.03
1,Eixample,54.29,13.81
2,Gràcia,50.12,3.58
3,Horta-Guinardó,32.69,2.61
4,Les Corts,34.05,2.47
5,Nou Barris,41.25,4.75
6,Sant Andreu,34.76,5.49
7,Sant Martí,32.96,4.44
8,Sants-Montjuic,19.44,1.83
9,Sarrià-Sant Gervasi,21.85,1.24
